# Trafic Light Classificator

In [ ]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

In [ ]:
%matplotlib inline

In [ ]:
fig = plt.figure()


## 1. Data Import

In [ ]:
X_train = []
x_label = []
for img_class, directory in enumerate(['carla_red', 'carla_yellow', 'carla_green', 'carla_none']):
    for i, file_name in enumerate(glob.glob("{}/*.jpg".format(directory))):
        file = cv2.imread(file_name)
        file = cv2.cvtColor(file, cv2.COLOR_BGR2RGB);
        resized = cv2.resize(file, (32,64))


        X_train.append(resized/255.)
        x_label.append(img_class)
        
        #if (i < 3):
        #    plt.imshow(rgb)
        #    plt.show()
        
X_train = np.array(X_train)
x_label = np.array(x_label)

In [ ]:
x_label

## 2. CNN

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from keras.utils.np_utils import to_categorical
from keras import losses, optimizers, regularizers

In [ ]:
categorical_labels = to_categorical(x_label)

In [ ]:
num_classes = 4
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(64, 32, 3), padding='same', activation='relu', kernel_initializer='random_uniform', kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPooling2D(2,2))
#Dropout(0.8)
#model.add(Conv2D(32, (3, 3), padding='same', activation='relu', kernel_initializer='random_uniform', kernel_regularizer=regularizers.l2(0.01)))
#model.add(MaxPooling2D(2,2))
Dropout(0.2)
model.add(Flatten())

#model.add(Dense(128, activation='relu', kernel_initializer='random_uniform', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(8, activation='relu', kernel_initializer='random_uniform', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
loss = losses.categorical_crossentropy
optimizer = optimizers.Adam()

In [ ]:
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

In [ ]:
model.fit(X_train, categorical_labels, batch_size=64, epochs=10, verbose=True, validation_split=0.1, shuffle=True)

In [ ]:
score = model.evaluate(X_train, categorical_labels, verbose=0)

In [ ]:
print(score)

In [ ]:
prediction = model.predict(X_train, verbose=1)
for i in range(len(prediction)):
    print (i, prediction[i], categorical_labels[i])

In [ ]:
plt.imshow(X_train[46])

In [ ]:
import h5py
model.save('tl_classifier_carla.h5')